In [2]:
%defaultDatasource jdbc:h2:mem:db

In [3]:
--Criando as tabelas principais através do repositório
DROP TABLE IF EXISTS Esquizofrenia;
DROP TABLE IF EXISTS TabelaDatabase2;

CREATE TABLE Esquizofrenia(
    nomePais VARCHAR(52),
    anoPesquisa int,
    porcentagemHomens FLOAT,
    porcentagemMulheres FLOAT,
    PRIMARY KEY(nomePais, anoPesquisa)
) AS SELECT
    Entity,
    Year, 
    MaleSchizophreniaPerCent,
    FemaleSchizophreniaPerCent
FROM CSVREAD('../../data/databasse5_prevalence-of-schizophrenia-in-males-vs-femalesv3.csv');

CREATE TABLE TabelaDatabase2(
    nomePais VARCHAR(52) NOT NULL,
    categoria VARCHAR(50) NOT NULL,
    indicador VARCHAR(85) NOT NULL,
    anoPesquisa VARCHAR(9),
    porcentagem VARCHAR(8),
    PRIMARY KEY(nomePais, indicador, anoPesquisa)
) AS SELECT
    Country,
    Category,
    Indicator,
    Year, 
    Value
FROM CSVREAD('../../data/database2_overweight_underweight.csv');
--indicadores: 
--Overweight (WHZ >+2 SD) in children 0-59 months (%), Underweight (WAZ <-2 SD) in children 0-59 months (%), 
--Low birth weight (<2500 g) (%), Overweight (BMI-for-age >+1 SD) in school-age children and adolescents 5-19 years (%),

In [28]:
SELECT nomePais, indicador, anoPesquisa, porcentagem FROM TabelaDatabase2;
SELECT * FROM Esquizofrenia;

In [4]:
--Normalizando os dados
DELETE FROM TabelaDatabase2 TB2 WHERE TB2.porcentagem = 'No data' OR TB2.porcentagem IS NULL;
DELETE FROM Esquizofrenia E WHERE E.porcentagemHomens IS NULL OR E.porcentagemMulheres IS NULL;

In [14]:
--Primeiros Selects (por categoria específica)
SELECT nomePais, anoPesquisa, CAST(porcentagem AS FLOAT) CriancasComSobrepesoPorcentagem FROM TabelaDatabase2 WHERE indicador = 'Overweight (WHZ >+2 SD) in children 0-59 months (%)' ORDER BY porcentagem DESC;


In [15]:
--Primeiros Selects (por categoria específica)
SELECT nomePais, anoPesquisa, CAST(porcentagem AS FLOAT) porcentagem FROM TabelaDatabase2 WHERE indicador = 'Low birth weight (<2500 g) (%)' AND anoPesquisa = 2000 ORDER BY porcentagem DESC;

In [6]:
--Tabela Juncao para armazenar dados de ambas as tabelas em uma só
DROP TABLE IF EXISTS Juncao; 
CREATE TABLE Juncao(
    nomePais VARCHAR(52) NOT NULL,
    anoPesquisa int,
    categoria VARCHAR(50) NOT NULL,
    indicador VARCHAR(85) NOT NULL,
    porcentagem VARCHAR(8),
    porcentagemHomensEsq FLOAT,
    porcentagemMulheresEsq FLOAT
);

In [7]:
--Inserindo nela os valores de cada tabela
INSERT INTO Juncao (nomePais, anoPesquisa, categoria, indicador, porcentagem, porcentagemHomensEsq, porcentagemMulheresEsq)
SELECT TB2.nomePais, E.anoPesquisa, TB2.categoria, TB2.indicador, TB2.porcentagem, E.porcentagemHomens, E.porcentagemMulheres
FROM TabelaDatabase2 TB2, Esquizofrenia E WHERE TB2.nomePais = E.nomePais AND CAST(SUBSTRING(TB2.anoPesquisa, 1, 4) AS INT) = E.anoPesquisa;

In [18]:
--Nesta View deixaremos os cálculos com um certo arredondamento e colocaremos o indicador para este específico 'Overweight', assim todas as análises
--estarão, por enquanto, levando em conta somente este indicador.
DROP VIEW IF EXISTS Teste01;
CREATE VIEW TESTE01 AS
SELECT nomePais, anoPesquisa, CAST(porcentagem AS FLOAT) overweightPercentage, ROUND(porcentagemHomensEsq, 3) AS HomensEsqPercent, ROUND(porcentagemMulheresEsq, 3) AS MulheresEsqPercent FROM Juncao
WHERE indicador = 'Overweight (WHZ >+2 SD) in children 0-59 months (%)' AND anoPesquisa > 2010 ORDER BY overweightPercentage DESC;
SELECT * FROM TESTE01;

In [36]:
--Primeira análise
DROP TABLE IF EXISTS Caso01;
CREATE TABLE Caso01(
    nomeRange VARCHAR(25),
    porcentagemOWMedia DOUBLE,
    porcentagemEsqHomemMedia DOUBLE,
    porcentagemEsqMulherMedia DOUBLE
);

In [37]:
--Nesta primeira análise da relação entre tabelas, após explorar as tabelas individualmente, mostra que já podemos ver uma pequena relação entre
--a porcentagem de crianças com sobrepeso e a quantidade de casos de esquizofrenia em homens/mulheres:
--Assim, os divido em 3 diferentes categorias: High Child O., que é o grupo contendo uma porcentagem de O.W. maior do que 10 (países acima deste número estavam na parte de cima da tabela),
--Medium Child O., correspondente a 10 > porcentagem de O.W. > 5, que corresponde aos países com taxas médias de sobrepeso em crianças,
--Low Child O., corresponden
INSERT INTO Caso01 (porcentagemOWMedia, porcentagemEsqHomemMedia, porcentagemEsqMulherMedia)
SELECT ROUND(AVG(T01.overweightPercentage), 4), ROUND(AVG(T01.HomensEsqPercent), 4), ROUND(AVG(T01.MulheresEsqPercent), 4) FROM Teste01 T01 
WHERE T01.overweightPercentage > 10;
UPDATE Caso01 SET nomeRange = 'High Child Overweight' WHERE porcentagemOWMedia > 10;

INSERT INTO Caso01 (porcentagemOWMedia, porcentagemEsqHomemMedia, porcentagemEsqMulherMedia)
SELECT ROUND(AVG(T01.overweightPercentage), 4), ROUND(AVG(T01.HomensEsqPercent), 4), ROUND(AVG(T01.MulheresEsqPercent), 4) FROM Teste01 T01 
WHERE T01.overweightPercentage > 5 AND T01.overweightPercentage < 10;
UPDATE Caso01 SET nomeRange = 'Medium Child Overweight' WHERE porcentagemOWMedia > 5 AND porcentagemOWMedia < 10;

INSERT INTO Caso01 (porcentagemOWMedia, porcentagemEsqHomemMedia, porcentagemEsqMulherMedia)
SELECT ROUND(AVG(T01.overweightPercentage), 4), ROUND(AVG(T01.HomensEsqPercent), 4), ROUND(AVG(T01.MulheresEsqPercent), 4) FROM Teste01 T01 
WHERE T01.overweightPercentage < 5;
UPDATE Caso01 SET nomeRange = 'Low Child Overweight' WHERE porcentagemOWMedia < 5;

In [39]:
SELECT * FROM Caso01;